## Задача 1
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId


In [42]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')
rating_count = ratings.groupby('userId').count()
over_one_hundred = rating_count[rating_count['movieId'] > 100].reset_index()
over_one_hundred.head(10)

,userId,movieId,rating,timestamp
0,4,204,204,204
1,8,116,116,116
2,15,1700,1700,1700
3,17,363,363,363
4,19,423,423,423
5,21,162,162,162
6,22,220,220,220
7,23,726,726,726
8,26,172,172,172
9,30,1011,1011,1011


In [43]:
list_of_over100 = over_one_hundred['userId'].tolist()
list_of_over100[:10]

[4, 8, 15, 17, 19, 21, 22, 23, 26, 30]

In [44]:
def differ(x):
    return x['max'] - x['min']

rating100 = ratings[ratings['userId'].isin(list_of_over100)]
rating_minmax = rating100.groupby('userId').agg(['min','max'])['timestamp'].reset_index()
rating_minmax['lifetime'] = rating_minmax.apply(differ,axis=1)
rating_minmax.head()

,userId,min,max,lifetime
0,4,949778714,949982274,203560
1,8,1154389340,1154474527,85187
2,15,997937239,1469330735,471393496
3,17,1127468587,1127476640,8053
4,19,855190091,855195373,5282


In [45]:
rating_minmax['lifetime'].mean()

40080507.4496124

## Задача 2
Дана статистика услуг перевозок клиентов компании по типам. 

-rzd - железнодорожные перевозки

-auto - автомобильные перевозки

-air - воздушные перевозки

-client_base - адреса клиентов

Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [46]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [47]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [48]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [49]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [50]:
revenue_all = (rzd.merge(auto,on='client_id',how = 'outer')).merge(air,on='client_id',how='outer')
revenue_all

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [51]:
client_base.merge(revenue_all,how='left')

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


## Задача 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. 

Т. е. для каждого user_id известна последовательность координат (широта/долгота), 

когда они требовались приложению для полноценной работы.

Как бы вы добавили эти сведения в таблицу визитов и покупок?

## Ответ
Ответ: лучше всего эти две таблицы хранить раздельно, но если очень нужно соединяем их по user_id, лучше использовать таблицу 
    purchases_pivot,чтобы не дублировать данные. Чтобы не потерять данные при соединении  указываем тип соединения 'outer'
    или 'left', 'right', смотря что нам нужно.